In [ ]:
!pip install transformers torch faiss-cpu tqdm sqlalchemy

In [ ]:
import sqlite3, json
import numpy as np
import faiss
import torch
from transformers import AutoTokenizer, AutoModel

In [ ]:
# ---------------------------
# Config
# ---------------------------
MODEL_NAME = "src/notebook/Data_Fine_Tune/BAAI/bge-m3"
MAX_LENGTH = 512
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------------------
# Load model/tokenizer
# ---------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE).eval()


In [ ]:
# ---------------------------
# Embed query
# ---------------------------
@torch.no_grad()
def embed_query(q: str) -> np.ndarray:
    if not q or not q.strip():
        return np.zeros((1, model.config.hidden_size), dtype="float32")
    enc = tokenizer(q, padding=True, truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
    enc = {k: v.to(DEVICE) for k,v in enc.items()}
    out = model(**enc)
    last_hidden = out.last_hidden_state
    mask = enc["attention_mask"].unsqueeze(-1).expand(last_hidden.size()).float()
    summed = torch.sum(last_hidden * mask, dim=1)
    counts = torch.clamp(mask.sum(dim=1), min=1e-9)
    mean = summed / counts
    arr = mean.cpu().numpy()
    arr = arr / (np.linalg.norm(arr, axis=1, keepdims=True) + 1e-9)
    return arr.astype("float32")

In [ ]:
# ---------------------------
# Load FAISS
# ---------------------------
def load_index(faiss_file: str):
    return faiss.read_index(faiss_file)

In [ ]:
# ---------------------------
# Fetch docs
# ---------------------------
def fetch_docs_by_faiss_ids(conn, faiss_ids):
    if not faiss_ids:
        return []
    q_marks = ",".join("?" * len(faiss_ids))
    rows = conn.execute(
        f"SELECT faiss_id, uuid, doc_text, metadata_json FROM vectors WHERE faiss_id IN ({q_marks})",
        tuple(faiss_ids)
    ).fetchall()
    rows_map = {r[0]: {"uuid": r[1], "text": r[2], "metadata": json.loads(r[3])} for r in rows}
    return [rows_map[i] for i in faiss_ids if i in rows_map]


In [ ]:
# ---------------------------
# RAG search
# ---------------------------
def rag_search(query: str, faiss_file="./database/laws_bge.index", db="./database/laws_bge.db", top_k=5):
    conn = sqlite3.connect(db)
    index = load_index(faiss_file)
    q_emb = embed_query(query)
    D, I = index.search(q_emb, top_k)
    ids = I[0].tolist()
    docs = fetch_docs_by_faiss_ids(conn, ids)
    conn.close()
    return [{"score": float(D[0][idx]), **docs[idx]} for idx in range(len(docs)) if idx < len(docs)]

In [ ]:
# ---------------------------
# Example
# ---------------------------
if __name__ == "__main__":
    s = "Khoản 1 Điều 8 luật hôn nhân"
    results = rag_search(s, top_k=5)
    for i, r in enumerate(results):
        print(f"Rank {i+1} | score={r['score']:.4f} | uuid={r['uuid']}")
        print(r['text'])
        print("metadata:", r['metadata'])
        print("----")

In [ ]:
# Colab: cài thư viện (chọn faiss-gpu nếu bạn chắc driver tương thích)
!pip install -q transformers sentence-transformers faiss-cpu torch torchvision torchaudio --upgrade

In [ ]:
# ==========================================================
# Mục tiêu:
#   So sánh hiệu suất giữa:
#     1. Kết quả truy hồi ban đầu từ FAISS (embedding BGE)
#     2. Kết quả sau khi rerank bằng model BGE-reranker-v2-m3
#   Đánh giá bằng MRR@K và nDCG@K
# ==========================================================

import math, re, sqlite3, sys, time
import pandas as pd, numpy as np
from typing import List

# ========== CẤU HÌNH CƠ BẢN ==========

# File test chứa danh sách câu hỏi (query) và câu trả lời đúng (ground truth)
TEST_FILE = "src/notebook/Data_Fine_Tune/test.xlsx"
TEST_QUERY_COL = "câu hỏi"          # Tên cột chứa câu hỏi
TEST_RELEVANT_COL = "câu trả lời"   # Tên cột chứa câu trả lời đúng
TEST_RELEVANT_IS_UUID = False       # Nếu ground truth là ID trong DB thì đặt True

# Cấu hình đánh giá
TOPK_EVAL = 5                       # Đánh giá trong top 5
FAISS_CANDIDATES_K = 30             # Lấy top 30 kết quả đầu tiên từ FAISS
RERANK_TOPK = 5                     # Sau khi rerank, chỉ lấy top 5 để đánh giá

# Đường dẫn FAISS index và database text
FAISS_FILE = "./database/laws_bge.index"
DB_FILE = "./database/laws_bge.db"

# Model reranker sử dụng (dùng từ HuggingFace)
RERANK_MODEL = "BAAI/bge-reranker-v2-m3"

# Cấu hình batch khi rerank
RERANK_BATCH_SIZE = 16              # Số cặp query-doc xử lý cùng lúc
RERANK_MAX_LENGTH = 512             # Giới hạn token để tránh lỗi quá dài


In [ ]:
# PHẦN 1. Load mô hình reranker

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Dùng GPU nếu có, nếu không dùng CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device for reranker:", device)

# Tải tokenizer và model từ HuggingFace
try:
    rerank_tokenizer = AutoTokenizer.from_pretrained(RERANK_MODEL)
    rerank_model = AutoModelForSequenceClassification.from_pretrained(RERANK_MODEL).to(device).eval()
except Exception as e:
    raise RuntimeError(f"Không thể load reranker '{RERANK_MODEL}'. Cần mạng hoặc model không tồn tại: {e}")

In [ ]:
# PHẦN 2. Các hàm tiện ích tiền xử lý & so sánh text

def normalize_text(s: str) -> str:
    """Chuẩn hóa text:
       - Chuyển về chữ thường
       - Loại bỏ khoảng trắng thừa"""
    if s is None: return ""
    s = str(s).lower()
    return re.sub(r'\s+', ' ', s).strip()

def parse_relevant_cell(cell, is_uuid=False):
    """Tách các câu trả lời đúng trong 1 ô Excel.
       Có thể có nhiều câu trả lời ngăn cách bởi dấu | hoặc ; hoặc xuống dòng"""
    if pd.isna(cell): return set()
    if is_uuid:
        # Nếu câu trả lời là ID trong DB
        s = str(cell)
        parts = [p.strip() for p in re.split(r'[,\|;]+', s) if p.strip()]
        return set(parts)
    # Nếu câu trả lời là text
    s = str(cell).strip()
    parts = [p.strip() for p in re.split(r'\r\n|\n|\|\||;|\|', s) if p.strip()]
    return set(parts)

def match_by_text(returned_text: str, ground_truth_texts: set, token_overlap_thresh=0.6) -> bool:
    """Kiểm tra xem văn bản trả về có khớp với ground truth không
       Dựa trên tỉ lệ trùng token (mặc định ≥ 60%)"""
    rt = normalize_text(returned_text)
    if rt == "": return False
    for gt in ground_truth_texts:
        gt_n = normalize_text(gt)
        if gt_n == "": continue

        # Nếu một chuỗi nằm trong chuỗi kia → coi là trùng
        if gt_n in rt or rt in gt_n: return True

        # Nếu trùng ≥ 60% token → coi là trùng
        rt_tokens = set(rt.split()); gt_tokens = set(gt_n.split())
        if len(gt_tokens) == 0: continue
        inter = rt_tokens.intersection(gt_tokens)
        if len(inter) / len(gt_tokens) >= token_overlap_thresh: return True
    return False

In [ ]:
# PHẦN 3. Hàm tính các metric đánh giá

def rr_at_k_list(ranked_list, relevant_set, k):
    """Tính Reciprocal Rank (RR):
       = 1 / vị trí của phần tử đúng đầu tiên trong top-k"""
    for i, item in enumerate(ranked_list[:k], start=1):
        if item in relevant_set: return 1.0 / i
    return 0.0

def dcg_at_k_binary(ranked_list, relevant_set, k):
    """Tính DCG@k (Discounted Cumulative Gain)
       1 nếu item đúng, chia cho log2(vị trí + 1)"""
    dcg = 0.0
    for i, item in enumerate(ranked_list[:k], start=1):
        if item in relevant_set:
            dcg += 1.0 / math.log2(i + 1)
    return dcg

def idcg_binary(n_relevant, k):
    """Tính IDCG@k (DCG lý tưởng) để chuẩn hóa thành nDCG"""
    idcg = 0.0
    for i in range(1, min(n_relevant, k) + 1):
        idcg += 1.0 / math.log2(i + 1)
    return idcg


In [ ]:
# PHẦN 4. Reranker sử dụng model Sequence Classification

@torch.no_grad()
def rerank_scores_bge_classifier(query: str, docs: List[str],
                                 batch_size:int = RERANK_BATCH_SIZE,
                                 max_length:int = RERANK_MAX_LENGTH):
    """
    Hàm tính điểm relevance giữa query và danh sách documents.
    - Input: query (str), docs (list[str])
    - Output: mảng numpy các điểm (float32), càng cao → càng liên quan
    """
    scores = []
    for i in range(0, len(docs), batch_size):
        # Lấy 1 batch docs
        batch_docs = docs[i:i+batch_size]

        # Tokenizer chấp nhận list cặp (query, doc)
        enc = rerank_tokenizer([query]*len(batch_docs), batch_docs,
                                padding=True, truncation=True, max_length=max_length,
                                return_tensors="pt")
        enc = {k: v.to(device) for k,v in enc.items()}  # Chuyển sang GPU/CPU
        out = rerank_model(**enc)                       # Chạy model
        logits = out.logits.detach().cpu().numpy()      # Kết quả (B, num_labels)

        # Nếu model chỉ có 1 logit → lấy trực tiếp
        if logits.shape[1] == 1:
            batch_scores = logits[:,0].tolist()
        else:
            # Nếu có 2 lớp → lấy điểm lớp "positive" (index 1)
            batch_scores = logits[:,1].tolist()

        scores.extend(batch_scores)
    return np.array(scores, dtype="float32")


In [ ]:
# PHẦN 5. CHẠY CHÍNH: đọc file test, truy hồi FAISS, rerank và đánh giá

if TEST_FILE.lower().endswith((".xls", ".xlsx")):
    df = pd.read_excel(TEST_FILE)
else:
    df = pd.read_csv(TEST_FILE)

# Kiểm tra cột hợp lệ
if TEST_QUERY_COL not in df.columns or TEST_RELEVANT_COL not in df.columns:
    print("LỖI: Không tìm thấy cột TEST_QUERY_COL hoặc TEST_RELEVANT_COL trong file test.")
    print("Cột có trong file:", df.columns.tolist())
    sys.exit(1)

# Lấy danh sách câu hỏi và câu trả lời đúng
queries = df[TEST_QUERY_COL].astype(str).tolist()
relevant_cells = df[TEST_RELEVANT_COL].tolist()
parsed_relevants = [parse_relevant_cell(c, is_uuid=TEST_RELEVANT_IS_UUID) for c in relevant_cells]

# --- Load FAISS index ---
try:
    index = load_index(FAISS_FILE)
except Exception as e:
    raise RuntimeError(f"Không thể load FAISS index từ {FAISS_FILE}: {e}")

# --- Biến lưu kết quả đánh giá ---
faiss_rrs = []; faiss_ndcgs = []
rerank_rrs = []; rerank_ndcgs = []

# --- Vòng lặp qua từng câu hỏi ---
for i, q in enumerate(queries):
    print(f"[{i+1}/{len(queries)}] Query: {q[:80]}...")
    gt_set = parsed_relevants[i]  # ground truth cho query này

    # ===== FAISS RETRIEVAL =====
    q_emb = embed_query(q)                                # Tạo embedding cho query
    D, I = index.search(q_emb, FAISS_CANDIDATES_K)        # Lấy top 30 văn bản gần nhất
    faiss_ids = I[0].tolist()

    # Lấy text của các văn bản từ database
    conn = sqlite3.connect(DB_FILE)
    faiss_docs = fetch_docs_by_faiss_ids(conn, faiss_ids)
    conn.close()

    # ===== ĐÁNH GIÁ FAISS =====
    if TEST_RELEVANT_IS_UUID:
        # Nếu ground truth là ID
        faiss_ranked = [d['uuid'] for d in faiss_docs][:TOPK_EVAL]
        faiss_rr = rr_at_k_list(faiss_ranked, gt_set, TOPK_EVAL)
        dcg = dcg_at_k_binary(faiss_ranked, gt_set, TOPK_EVAL)
        idcg = idcg_binary(len(gt_set), TOPK_EVAL)
        faiss_ndcg = (dcg / idcg) if idcg>0 else 0.0
    else:
        # Nếu ground truth là text
        faiss_ranked_texts = [normalize_text(d['text']) for d in faiss_docs][:TOPK_EVAL]
        rels = [1 if match_by_text(t, gt_set) else 0 for t in faiss_ranked_texts]

        # Tính RR và nDCG
        faiss_rr = 0.0
        for pos, v in enumerate(rels, start=1):
            if v:
                faiss_rr = 1.0/pos; break
        dcg = sum((1.0 / math.log2(pos+1)) for pos, v in enumerate(rels, start=1) if v)
        idcg = idcg_binary(len(gt_set), TOPK_EVAL)
        faiss_ndcg = (dcg / idcg) if idcg>0 else 0.0

    # ===== RERANKER =====
    docs_texts = [d['text'] for d in faiss_docs]  # Lấy text của các docs từ FAISS

    if len(docs_texts) == 0:
        # Nếu FAISS không trả về gì
        rerank_rr = 0.0
        rerank_ndcg = 0.0
    else:
        try:
            # Tính điểm liên quan (relevance score)
            scores = rerank_scores_bge_classifier(q, docs_texts,
                                                  batch_size=RERANK_BATCH_SIZE,
                                                  max_length=RERANK_MAX_LENGTH)
        except Exception as e:
            print(f"Reranker failed for query {i}: {e}")
            scores = np.zeros(len(docs_texts), dtype="float32")

        # Sắp xếp docs theo điểm giảm dần
        ranked_idx_scores = sorted(list(enumerate(scores)), key=lambda x: x[1], reverse=True)
        topk_idx = [idx for idx, sc in ranked_idx_scores[:RERANK_TOPK]]

        # ===== ĐÁNH GIÁ SAU RERANK =====
        if TEST_RELEVANT_IS_UUID:
            rerank_ranked = [faiss_docs[idx]['uuid'] for idx in topk_idx]
            rerank_rr = rr_at_k_list(rerank_ranked, gt_set, TOPK_EVAL)
            dcg = dcg_at_k_binary(rerank_ranked, gt_set, TOPK_EVAL)
            idcg = idcg_binary(len(gt_set), TOPK_EVAL)
            rerank_ndcg = (dcg / idcg) if idcg>0 else 0.0
        else:
            rerank_topk_texts = [normalize_text(docs_texts[idx]) for idx in topk_idx]
            rels = [1 if match_by_text(t, gt_set) else 0 for t in rerank_topk_texts]
            rerank_rr = 0.0
            for pos, v in enumerate(rels, start=1):
                if v:
                    rerank_rr = 1.0/pos; break
            dcg = sum((1.0 / math.log2(pos+1)) for pos, v in enumerate(rels, start=1) if v)
            idcg = idcg_binary(len(gt_set), TOPK_EVAL)
            rerank_ndcg = (dcg / idcg) if idcg>0 else 0.0

    # Lưu kết quả từng query
    faiss_rrs.append(faiss_rr)
    faiss_ndcgs.append(faiss_ndcg)
    rerank_rrs.append(rerank_rr)
    rerank_ndcgs.append(rerank_ndcg)

In [ ]:
print("SUMMARY")
print("FAISS-only MRR@{} = {:.6f}, nDCG@{} = {:.6f}".format(
    TOPK_EVAL, np.mean(faiss_rrs), TOPK_EVAL, np.mean(faiss_ndcgs)))
print(f"RERANKER ({RERANK_MODEL}) MRR@{TOPK_EVAL} = {np.mean(rerank_rrs):.6f}, nDCG@{TOPK_EVAL} = {np.mean(rerank_ndcgs):.6f}")